# Link Analysis
---
This notebook creates a retweet graph and runs different algorithms to predict links between a sample of nodes. 

The algorithms used to predict the links are Personalized Pagerank, Adamic-Adar, ALS and Jaccard Similarity. 

All of them are compared in terms of Accuracy.

## Imports

In [1]:
import pandas as pd
import json
import igraph as ig
import matplotlib.pyplot as plt
import random
import numpy as np
import itertools 
import implicit

## Create Graph

In [2]:
df_retweets = pd.read_csv("retweets.csv")
df_graph = pd.DataFrame(columns=["source", "destination"])
df_retweets.dropna(inplace=True)

# add source-nodes
df_graph["source"] = df_retweets['user']

# add destination-nodes
df_graph["destination"] = df_retweets['original_text'].apply(lambda x: x.split()[1][1:])
df_graph.drop_duplicates(inplace=True)
df_graph.head()

df_graph.reset_index(drop=True,inplace=True)

# Free memory
del df_retweets

In [3]:
tuples = [tuple(x) for x in df_graph.values]
graph = ig.Graph.TupleList(tuples, directed = False, edge_attrs = ['weight'])
original_graph = graph.copy()

## Splitting

In [4]:
# fraction of edges to select as test-set
test_size = 0.2

# graph size
N = len(graph.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
T = np.random.choice(a=all_idxs, size=int(test_size*N), replace=False)

In [5]:
# Remove test_idxs from graph
graph.delete_edges(T)

In [6]:
# Compute groundtruth (U) and training set
U = set() #--> set that contain the real values
trainset = set() #--> to predict values
for idx, one_edge in enumerate(original_graph.es):
    
    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target

    if idx in T:
        U.add((n1, n2, 1))
    else:
        trainset.add((n1, n2, 1))

In [7]:
test_nodes_idxs = set()

# Obtain test nodes ids
for n1, n2, edge in U:
    test_nodes_idxs.add(n1)
    test_nodes_idxs.add(n2)

len(test_nodes_idxs)

17221

In [8]:
def find_nodes_at_distance_2(graph, test_nodes, num_neighbors):
    """
    starting from a graph this function returns all the nodes at distance 2 from the test nodes
    """
    
    all_potential_recommendations = set()
    for n1 in test_nodes:   
        
        # all the nodes at distance 1
        nodes_at_most_distant_1 = set(graph.neighborhood(vertices=n1, order=1)) # --> Node Ids at distance 1 from node n1

        # all the nodes at distance 1 and distance 2
        nodes_at_most_distant_2 = set(graph.neighborhood(vertices=n1, order=2))
        
        # only the nodes at distance 2
        only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1
        
        # check if empty set
        if len(only_nodes_at_distance_2) > 0:

            for n2 in only_nodes_at_distance_2:
                
                # n1 is an ID of an igraph vertex
                n1_index = n1.index 
                
                if len(all_potential_recommendations) == num_neighbors:
                    return all_potential_recommendations
                
                all_potential_recommendations.add((n1_index, n2)) 

    return all_potential_recommendations

In [9]:
# Find len(T) nodes at distance 2 from some node of the test set
D2 = find_nodes_at_distance_2(graph,graph.vs[test_nodes_idxs], num_neighbors = len(T))

# Number of neighbours of source test nodes at distance 2 from all the graph
print('Number of neighbours at distance 2 of the test set nodes:',len(D2))

Number of neighbours at distance 2 of the test set nodes: 13602


In [10]:
# Add potential nodes to ground truth
for rec in D2:
    n1 = rec[0]
    n2 = rec[1]
    
    U.add((n1,n2,0))

In [11]:
test_sample_ground_truth = set()
test_nodes_idxs = list(test_nodes_idxs)
for n1, n2, val in U:
    if n1 in test_nodes_idxs or n2 in test_nodes_idxs:
        test_sample_ground_truth.add((n1, n2, val))

## Algorithms

## Pagerank

In [12]:
# Recomend for each node in list_nodes the topK node (IDs) sorted by personalized pagerank
def personalized_pagerank(graph, node, k):
    
    # Only keep pagerank values of other nodes
    pr = [i for i in enumerate(graph.personalized_pagerank(reset_vertices = node)) if i[0] != node]
    
    # Sort by decreasing pagerank
    out = sorted(pr, key=lambda tup: tup[1], reverse=True)[:k]
    
    return out

In [13]:
k = 10

# Store pageranks of nodes in U in a dictionary
PR_dict = {}
for n1, _, _ in U:  
    if n1 not in PR_dict.keys():
        PR_dict[n1] = personalized_pagerank(graph, n1, k)

In [14]:
df_PR = pd.DataFrame(columns=['n1','n2','edge','pred_PR'])

# Compute pagerank for each pair of nodes of U
for n1, n2, edge in U:  
    pred_PR_list = [i[1] for i in PR_dict[n1] if i[0] == n2]
    
    # If list empty pagerank values is 0
    if len(pred_PR_list) != 0:
        pred_PR = pred_PR_list[0]
    else:
        pred_PR = 0
    df_PR = df_PR.append({'n1': n1, 'n2': n2, 'edge': edge,'pred_PR': pred_PR}, ignore_index = True)

In [15]:
df_PR.pred_PR = df_PR.pred_PR.apply(lambda x: round(x))

PR_accuracy = len(df_PR[df_PR['pred_PR'] == df_PR['edge']])/len(U)

## Adamic-Adar

In [16]:
def compute_ADA(u,v, graph):
    """
    compute adamic-adar from scratch
    """  
    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v = graph.neighbors(v)

    # set Z of neighbors of both --> set Z = intersection of neighbors(nodes)
    bridges = set(outlinks_from_u).intersection(inlinks_to_v)

    # degree of nodes in set Z
    deg_ = [graph.degree(n) for n in bridges] #--> looping over all the nodes of the set bridge
    
    # computing the reciprocal in log-scale
    out = [1./np.log2(dd+1) for dd in deg_]

    return sum(out)

In [17]:
df_ADA = pd.DataFrame(columns=['n1','n2','edge','pred_ADA'])

# Compute Adamic-Adar between all pairs of nodes of U
for n1, n2, edge in U:  
    df_ADA = df_ADA.append({'n1': n1, 'n2': n2, 'edge': edge,'pred_ADA': compute_ADA(n1, n2, graph)}, ignore_index = True)

In [18]:
df_ADA.pred_ADA = df_ADA.pred_ADA.apply(lambda x: round(x))

ADA_accuracy = len(df_ADA[df_ADA['pred_ADA'] == df_ADA['edge']])/len(U)

## Jaccard

In [43]:
def compute_Jaccard(u,v, graph):
    """
    compute jaccard similarity
    """
    # set of neighbors of u
    outlinks_from_u = graph.neighbors(u)

    # set of neighbors of v
    inlinks_to_v =graph.neighbors(v)

    # intesection of the two sets
    num = set(outlinks_from_u).intersection(inlinks_to_v)
    
    # union of the two sets
    den = set(outlinks_from_u).union(inlinks_to_v)
    
    if len(den) == 0:
        return 0
    
    # final division
    out = len(num)/len(den)
    
    return out

In [38]:
df_Jaccard = pd.DataFrame(columns=['n1','n2','edge','pred_Jaccard'])

# Compute Jaccard similarity between all pairs of nodes of U
for n1, n2, edge in U:  
    df_Jaccard = df_Jaccard.append({'n1': n1, 'n2': n2, 'edge': edge,'pred_Jaccard': compute_Jaccard(n1, n2, graph)},
                                   ignore_index = True)

In [39]:
df_Jaccard.pred_Jaccard = df_Jaccard.pred_Jaccard.apply(lambda x: round(x))

#accuracy
Jaccard_accuracy = len(df_Jaccard[df_Jaccard.edge == df_Jaccard.pred_Jaccard])/len(U)

## ALS

In [57]:
# first we get the adjacency matrix data
M = graph.get_adjacency_sparse()

# here we run the model ALS
model = implicit.als.AlternatingLeastSquares(factors=250, calculate_training_loss=True,  iterations=10, random_state= 123)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(M)

In [58]:
def predict_ALS(testset, model):
    """
    predict for a list of observations the score for adding/removing a link
    """

    # initialize the empty list
    all_predictions = []

    # scroll the obs
    for n1, n2, w in testset:
        
        # take here the low-dimensional vectors returned by the matrix factorization
        
        array_n1 = model.user_factors[n1,:]
        array_n2 = model.item_factors[n2,:]

        # multiplying these vectors we generate an approximation for the edge score
        one_p = np.dot(array_n1, array_n2)

        all_predictions.append(one_p)
        
    return all_predictions

In [59]:
# generate the predictions
df_test = pd.DataFrame(list(U), columns=["n1","n2", "edge"])
all_predictions = predict_ALS(df_test.values, model)

# add predictions to df
df_test["rating"] = all_predictions

# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test["rating"] = df_test["rating"].apply(lambda x: int(x >= 0.5))

In [60]:
# number of observations matched by the prediction
right_predictions = len(df_test[df_test["rating"] == df_test["edge"]])

# accuracy 
ALS_accuracy = right_predictions/len(df_test)

## RQ 3A

In [61]:
print('Pagerank Accuracy:',PR_accuracy)
print('Adamic-Adar Accuracy:',ADA_accuracy)
print('Jaccard Accuracy:',Jaccard_accuracy)
print('ALS Accuracy:',ALS_accuracy)

Pagerank Accuracy: 0.5
Adamic-Adar Accuracy: 0.4555212468754595
Jaccard Accuracy: 0.4215188942802529
ALS Accuracy: 0.5
